# ICU Multimodal Patient Monitoring: Static PID vs Temporal Directed Information

In [2]:
import math
from sklearn.metrics import mutual_info_score

# Binary flags for HR, BP, and event from the scenario table
HR_flag = [1, 0, 1, 0]    # 1=HR high, 0=normal
BP_flag = [1, 1, 0, 0]    # 1=BP low, 0=normal
Event   = [1, 0, 0, 0]    # 1=shock event, 0=no event

# Mutual information (in bits) for single modalities and joint:
I_hr_y  = mutual_info_score(HR_flag, Event) / math.log(2)      # I(HR; Y)
I_bp_y  = mutual_info_score(BP_flag, Event) / math.log(2)      # I(BP; Y)
joint_state = [f"{h}{b}" for h,b in zip(HR_flag, BP_flag)]     # joint state of (HR,BP)
I_hrbp_y = mutual_info_score(joint_state, Event) / math.log(2) # I([HR,BP]; Y)

# PID decomposition (Williams & Beer 2010 using I_min redundancy)
I_red   = min(I_hr_y, I_bp_y)                        # redundant info
U_hr    = I_hr_y - I_red                             # HR unique info
U_bp    = I_bp_y - I_red                             # BP unique info
S       = I_hrbp_y - I_hr_y - I_bp_y + I_red         # synergistic info

print(f"I(HR;Y) = {I_hr_y:.3f} bits, I(BP;Y) = {I_bp_y:.3f} bits")
print(f"I(HR,BP;Y) = {I_hrbp_y:.3f} bits")
print(f"Redundancy = {I_red:.3f}, Unique(HR) = {U_hr:.3f}, Unique(BP) = {U_bp:.3f}, Synergy = {S:.3f}")

I(HR;Y) = 0.311 bits, I(BP;Y) = 0.311 bits
I(HR,BP;Y) = 0.811 bits
Redundancy = 0.311, Unique(HR) = 0.000, Unique(BP) = 0.000, Synergy = 0.500


In [3]:
# Simulated timeline (simplified): 0=normal, 1=anomalous
HR_flag_seq = [0, 0, 0, 0,  # baseline normal
               0,          # t=4: BP drop starts (will reflect in BP_flag_seq)
               0,          # HR normal until BP drops
               0,          # 
               0]          # (extend as needed)
BP_flag_seq = [0, 0, 0, 0, 
               1,          # t=4: BP_flag goes low (BP drop)
               1,          # t=5: BP still low
               1,          # t=6: BP low...
               0]          # recovery

# At t=5, HR responds to prior BP drop:
HR_flag_seq[5] = 1          # t=5: HR_flag goes high in response to BP low at t=4

# Calculate transfer entropy TE(BP -> HR)
import math
from collections import Counter
triples = Counter()
for t in range(1, len(HR_flag_seq)):
    triples[(BP_flag_seq[t-1], HR_flag_seq[t-1], HR_flag_seq[t])] += 1

# Compute joint and conditional probabilities
total = sum(triples.values())
TE_bp_to_hr = 0.0
# Sum_{bp_prev, hr_prev, hr_curr} P(bp_prev, hr_prev, hr_curr) * log2[ P(hr_curr | bp_prev, hr_prev) / P(hr_curr | hr_prev) ]
# (We derive conditional probabilities from frequency counts)
from collections import defaultdict
cond_counts = defaultdict(lambda: defaultdict(int))
cond_counts_hr = defaultdict(lambda: defaultdict(int))
for (bp_prev, hr_prev, hr_curr), count in triples.items():
    cond_counts[(hr_prev, bp_prev)][hr_curr] += count
    cond_counts_hr[hr_prev][hr_curr] += count

for (hr_prev, bp_prev), outcomes in cond_counts.items():
    for hr_curr, count in outcomes.items():
        p_joint = count / total
        p_hr_given_hr_bp = count / sum(outcomes.values())
        p_hr_given_hr = count / sum(cond_counts_hr[hr_prev].values())
        TE_bp_to_hr += p_joint * math.log2(p_hr_given_hr_bp / p_hr_given_hr)

print("DI (BP->HR) =", TE_bp_to_hr, "bits")

DI (BP->HR) = 0.7871107149038481 bits
